# Conversational RAG

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 langchain-groq langchain-google-genai langgraph

In [2]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.config import RunnableConfig
from langchain_groq import ChatGroq
from google.colab import userdata

In [3]:
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
LANGCHAIN_TRACING_V2 = userdata.get('LANGCHAIN_TRACING_V2')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [4]:
llm = ChatGroq(model="llama3-8b-8192", groq_api_key=GROQ_API_KEY, temperature=0)

In [5]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=GOOGLE_API_KEY
        )
)
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [6]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

"Task Decomposition is a process that breaks down a complicated task into smaller and simpler steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts (Yao et al. 2023), which help to decompose hard tasks into manageable tasks and provide an interpretation of the model's thinking process."

# Adding chat history

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [9]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Common ways of doing Task Decomposition include using Large Language Models (LLM) with simple prompting, task-specific instructions, or human inputs.


# Stateful management of chat history

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [11]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Task Decomposition is a process that breaks down a complicated task into smaller and simpler steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts (Yao et al. 2023), which help to decompose hard tasks into manageable tasks and provide an interpretation of the model's thinking process."

In [12]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Common ways of doing Task Decomposition include using Large Language Models (LLM) with simple prompting, task-specific instructions, or human inputs.'

In [13]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task Decomposition is a process that breaks down a complicated task into smaller and simpler steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts (Yao et al. 2023), which help to decompose hard tasks into manageable tasks and provide an interpretation of the model's thinking process.

User: What are common ways of doing it?

AI: Common ways of doing Task Decomposition include using Large Language Models (LLM) with simple prompting, task-specific instructions, or human inputs.



# Retrieval tool

In [14]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]

In [15]:
tool.invoke("task decomposition")

'Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority

# Agent constructor

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [17]:
query = "Explain the concept of Task Decomposition."

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_29cd', 'function': {'arguments': '{"query":"Task Decomposition"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 428, 'prompt_tokens': 929, 'total_tokens': 1357, 'completion_time': 0.356666667, 'prompt_time': 0.189511988, 'queue_time': None, 'total_time': 0.546178655}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8d441aa4-064c-4a77-b92a-5b75e12a1ee4-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'call_29cd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 428, 'total_tokens': 1357})]}}
----
{'tools': {'messages': [ToolMessage(content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An age

In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [19]:
config = RunnableConfig(configurable={"thread_id": "abc123"})

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob!', response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 946, 'total_tokens': 950, 'completion_time': 0.003333333, 'prompt_time': 0.142477789, 'queue_time': None, 'total_time': 0.145811122}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-3105e795-20db-417a-8a40-35f353308b5b-0', usage_metadata={'input_tokens': 946, 'output_tokens': 4, 'total_tokens': 950})]}}
----


In [20]:
query = "Could you please explain the concept of Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nvmq', 'function': {'arguments': '{"query":"Task Decomposition"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 972, 'total_tokens': 1043, 'completion_time': 0.055789873, 'prompt_time': 0.193758423, 'queue_time': None, 'total_time': 0.249548296}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d345ce34-a9d8-42b7-bfc2-65177d55289e-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'call_nvmq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 972, 'output_tokens': 71, 'total_tokens': 1043})]}}
----
{'tools': {'messages': [ToolMessage(content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent

In [21]:
query = "What according to the blog post are common ways of doing it? redo the search"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9f0m', 'function': {'arguments': '{"query":"What are common ways of doing task decomposition?"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 1760, 'total_tokens': 1807, 'completion_time': 0.039166667, 'prompt_time': 0.327519013, 'queue_time': None, 'total_time': 0.36668568}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-55c13fd1-305b-4777-b3c9-50fdb3614f12-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'What are common ways of doing task decomposition?'}, 'id': 'call_9f0m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1760, 'output_tokens': 47, 'total_tokens': 1807})]}}
----
{'tools': {'messages': [ToolMessage(content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning 